In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder

from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_absolute_error

import pickle
import joblib

In [2]:
df =pd.read_csv("../data/df_inner_join.csv", index_col=0)
df

,name,statff_cnt,cost_per_area,nums_of_franchisees,new_nums_of_franchisee,avr_sales,avr_sales_per_area,start_cost,category,year,debt_ratio,profit,fairness,growth,stability
0,이디야커피,424,209.0,2875,305,18704.0,657.0,12913.0,커피,21,88,4,3,3,3
1,비비큐,233,203.0,1746,270,59456.0,4388.0,9078.0,치킨,28,58,3,3,4,3
2,비에이치씨(BHC),167,225.0,1619,231,52103.0,2851.0,8543.0,치킨,19,205,5,3,4,4
3,배스킨라빈스,1816,390.0,1466,85,57170.0,3060.0,32335.0,아이스크림/빙수,37,24,4,3,3,2
4,투다리,16,298.0,1434,145,11251.0,790.0,4533.0,주점,35,126,5,3,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,다코기,1,165.0,6,0,31174.0,1743.0,8132.0,치킨,8,99,5,3,2,3
2550,밀크밥버거,65,220.0,6,0,2657.0,194.0,3630.0,분식,17,1292,2,3,3,5
2551,전현주식당,2,220.0,6,6,44344.0,1239.0,8836.0,한식,3,141,4,3,0,4
2552,육개장터,3,220.0,6,0,6168.0,394.0,5450.0,한식,6,-176,1,3,1,1


In [3]:
import numpy
numpy.mean(df['avr_sales'])

26523.00480769231

In [4]:
# 학습 / 평가 데이터셋을 분리합니다.
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)


# feature과 label을 분리합니다.

target = "avr_sales"
features = ["statff_cnt", "cost_per_area", "nums_of_franchisees", "new_nums_of_franchisee", "start_cost", "category", "year", "debt_ratio"]

X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]
X_val, y_val = val[features], val[target]
     

In [5]:
from category_encoders import OrdinalEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(
        objective="reg:squarederror", #objective="reg:squarederror"
        eval_metric="rmse",  #eval_metric="rmse"
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        max_depth=7,
       
        
    ),
)
print(pipe)



Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric='rmse',
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=7, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=200,
                              n_j

In [15]:
#하이퍼 파라미터 설정
from scipy.stats.distributions import uniform

params = {
    "xgbregressor__max_depth": [2,4,6,8],
    "xgbregressor__min_child_weight": [1, 3,5,7 , 10],
    "xgbregressor__colsample_bytree": uniform(
        loc=0.5, scale=0.5 
    ),
    'xgbregressor__gamma': [0.5, 1, 1.5, 2,3,4, 5],  
    'xgbregressor__subsample': [0.6,0.7, 0.8,0.9, 1.0],
    'xgbregressor__learning_rate':[0.01, 0.05, 0.1, 0.2]
}

In [16]:
#randomizedserchCV를 통해 하이퍼파라미터 탐색
from sklearn.model_selection import RandomizedSearchCV

randomized_search = RandomizedSearchCV(
    pipe,
    param_distributions=params,
    scoring="neg_mean_squared_error",
    n_iter=10,
    cv=3,
    verbose=3,
    random_state=42,
)
randomized_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END xgbregressor__colsample_bytree=0.6872700594236812, xgbregressor__gamma=3, xgbregressor__learning_rate=0.1, xgbregressor__max_depth=6, xgbregressor__min_child_weight=10, xgbregressor__subsample=1.0;, score=-204637917.557 total time=   0.4s
[CV 2/3] END xgbregressor__colsample_bytree=0.6872700594236812, xgbregressor__gamma=3, xgbregressor__learning_rate=0.1, xgbregressor__max_depth=6, xgbregressor__min_child_weight=10, xgbregressor__subsample=1.0;, score=-314575865.719 total time=   0.2s
[CV 3/3] END xgbregressor__colsample_bytree=0.6872700594236812, xgbregressor__gamma=3, xgbregressor__learning_rate=0.1, xgbregressor__max_depth=6, xgbregressor__min_child_weight=10, xgbregressor__subsample=1.0;, score=-238781249.261 total time=   0.2s
[CV 1/3] END xgbregressor__colsample_bytree=0.5780093202212182, xgbregressor__gamma=1.5, xgbregressor__learning_rate=0.1, xgbregressor__max_depth=6, xgbregressor__min_child_weight=5, 

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=['category'],
                                                             mapping=[{'col': 'category',
                                                                       'data_type': dtype('O'),
                                                                       'mapping': 기타 외식         1
치킨            2
분식            3
한식            4
제과제빵          5
일식            6
커피            7
음료 (커피 외)     8
주점            9
서양식          10
중식           11
피자           12
아이스크림/빙수     13
기타 외국식       14
패스트푸드        15
NaN          -2
dtype: int64}])),
                                             ('xgbregressor',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None...
                   param_distributions={'xgbregressor__colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f84408adc70>,
                                        'xgbregressor__gamma': [0.5, 1, 1.5, 2,
                                                                3, 4, 5],
                                        'xgbregressor__learning_rate': [0.01,
                                                                        0.05,
                                                                        0.1,
                                                                        0.2],
                                        'xgbregressor__max_depth': [2, 4, 6, 8],
                                        'xgbregressor__min_child_weight': [1, 3,
                                                                           5, 7,
                                                                           10],
                                        'xgbregressor__subsample': [0.6, 0.7,
                                                                    0.8, 0.9,
                                                                    1.0]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=3)

In [17]:
print("최적 하이퍼파라미터: ", randomized_search.best_params_)


최적 하이퍼파라미터:  {'xgbregressor__colsample_bytree': 0.7571172192068059, 'xgbregressor__gamma': 0.5, 'xgbregressor__learning_rate': 0.1, 'xgbregressor__max_depth': 2, 'xgbregressor__min_child_weight': 5, 'xgbregressor__subsample': 1.0}


In [30]:
#최적의 하이퍼파라미터 튜닝
pipe = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(
        objective="reg:squarederror",
        eval_metric="rmse",  
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        ###
        ###
        max_depth=2,
        learning_rate=0.1,
        min_child_weight=5,
        colsample_bytree=0.7571172192068059,
        gamma=0.5,
        subsample =1,
        
    ),
)
print(pipe)

        

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.7571172192068059,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric='rmse',
                              feature_types=None, gamma=0.5, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=2, max_leaves=None, min_child_weight=5,
                              missing=nan, monotone_constraints=None,
                              n_estimators

In [31]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['category'],
                                mapping=[{'col': 'category',
                                          'data_type': dtype('O'),
                                          'mapping': 기타 외식         1
치킨            2
분식            3
한식            4
제과제빵          5
일식            6
커피            7
음료 (커피 외)     8
주점            9
서양식          10
중식           11
피자           12
아이스크림/빙수     13
기타 외국식       14
패스트푸드        15
NaN          -2
dtype: int64}])),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_b...
                              feature_types=None, gamma=0.5, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=2, max_leaves=None, min_child_weight=5,
                              missing=nan, monotone_constraints=None,
                              n_estimators=200, n_jobs=-1,
                              num_parallel_tree=None, predictor=None,
                              random_state=42, ...))])

In [32]:
y_pred_train = pipe.predict(X_train)
y_pred_val = pipe.predict(X_val)

In [33]:
# Calculate MAE

rmse_pred = mean_absolute_error(y_val, y_pred_val) 
print("Root Mean Absolute Error:" , np.sqrt(rmse_pred))


Root Mean Absolute Error: 109.23602420906616


In [34]:
#모델저장하기
joblib.dump(pipe, open('sales_predict_model_new.model', 'wb')) 
# pickle.dump(pipe, open('sales_predict_model_new.model', 'wb')) 


In [37]:
model_pickle = joblib.load(open("sales_predict_model_new.model", 'rb'))


In [41]:
predicted_sales=model_pickle.predict(pd.DataFrame([[30,300,300,30,20000,'치킨', 5, 20]],
             columns =["statff_cnt", "cost_per_area", "nums_of_franchisees", "new_nums_of_franchisee", "start_cost", "category", "year", "debt_ratio"]
             )
             )
predicted_sales

array([42209.82], dtype=float32)

In [40]:
int(predicted_sales)

42209